En este notebook la idea es aprender lo básico de ASE, previo al trabajo con MACE. Para hacer esto me guiare de la documentación de ASE "https://ase-lib.org/gettingstarted/surface.html" donde hay unos pequeños scripts y un tutorial que te llevan de la mano a conocer que es ASE y como implementarlo para hacer simulaciones atómicas. 

Además se encontraron dos cursos básicos, a los cuales se les puede echar un ojo como material complementario a la práctica:

- https://ase-workshop-2023.github.io/tutorial/
- https://docs.matlantis.com/atomistic-simulation-tutorial/en/1_3_ase_basic.html

Empezaremos con la sección **Nitrogen on copper**

Previo a todo esto, me parece importante definir a ASE:

El "Atomic Simulation Enviroment" ASE por sus siglas, es una libreria de Python creada con el objetivo de configurar, dirigir y analizar simulaciones atomísticas. ASE ha sido construido con una serie de metas u objetivos.

1. **Facil de usar**: ASE puede ser empleado mediate una GUI (graphical user interface), a través de la linea de comandos (bash o CMD) y a través de python.
2. **Flexible**: La implementación de ASE en python la otorga de la flexiblidad del mismo lenguaje, el cual como herramienta logica permite diseñar rutinas que se acomodan al gusto y necesidad del usurio.
3. **Customizable**: ASE se estructura en modulos, cuyos objetivos varian y pueden ser implementados a conveniencia del usuario
4. **Pythonic**
5. **Abierto a participación**

### Introduction: Nitrogen on copper

This section gives a quick (and incomplete) overview of what ASE can do.

We will calculate the adsorption energy of a nitrogen molecule on a copper surface. This is done by calculating the total energy for the isolated slab and for the isolated molecule. The adsorbate is then added to the slab and relaxed, and the total energy for this composite system is calculated. The adsorption energy is obtained as the sum of the isolated energies minus the energy of the composite system.

Here is a picture of the system after the relaxation

![texto alternativo](./Imagenes/surface.png)


In [43]:
from ase import Atoms
from ase.build import add_adsorbate, fcc111
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton

h = 1.85
d = 1.10

slab = fcc111('Cu', size=(4, 4, 2), vacuum=10.0)

slab.calc = EMT()
e_slab = slab.get_potential_energy()
f_slab = slab.get_forces()

molecule = Atoms('2N', positions=[(0.0, 0.0, 0.0), (0.0, 0.0, d)])
molecule.calc = EMT()
e_N2 = molecule.get_potential_energy()
f_N2 = molecule.get_forces()

add_adsorbate(slab, molecule, h, 'ontop')
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)
dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

print('Adsorption energy:', e_slab + e_N2 - slab.get_potential_energy())

# print(f"Slab forces := {slab.get_forces()}")
print(f"Initial slab forces := {f_slab}")
print(f"Initial N2 forces := {f_N2}")

from ase.io import read
from ase.visualize import view

traj = read('N2Cu.traj', ':')   # dos puntos = leer toda la trayectoria
view(traj)                     # ver toda la trayectoria
view(traj[-1])                 # sólo la estructura final

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 17:00:06       11.689927       1.0797
BFGSLineSearch:    1[  2] 17:00:06       11.670814       0.4090
BFGSLineSearch:    2[  4] 17:00:06       11.625880       0.0409
Adsorption energy: 0.32351942231763964
Initial slab forces := [[-1.17983558e-15  4.16863972e-15  1.60676480e-01]
 [ 2.23210604e-15 -3.83164480e-15  1.60676480e-01]
 [ 2.13024043e-15  3.42781359e-15  1.60676480e-01]
 [ 1.77548948e-15 -6.31092401e-15  1.60676480e-01]
 [ 7.10802944e-16  3.20663635e-15  1.60676480e-01]
 [-7.70217223e-16  6.34388375e-15  1.60676480e-01]
 [ 3.84950339e-15  2.30024333e-15  1.60676480e-01]
 [ 1.79327039e-15  2.97331604e-15  1.60676480e-01]
 [ 1.59594560e-15 -1.97411532e-15  1.60676480e-01]
 [ 3.18495230e-15 -1.46323925e-15  1.60676480e-01]
 [ 2.70616862e-16 -1.58033309e-15  1.60676480e-01]
 [-8.88178420e-16 -2.66453526e-15  1.60676480e-01]
 [ 4.26741975e-15 -2.86055901e-15  1.60676480e-01]
 [ 7.45264162e-15 

<Popen: returncode: None args: ['c:\\Users\\danie\\AppData\\Local\\Programs\...>

Replica empleando mace.calculators : mace_mp

In [42]:
from mace.calculators import mace_mp

calculo = mace_mp(model="small", device="cpu", default_dtype="float64")

h = 1.85
d = 1.10

slab = fcc111('Cu', size=(4, 4, 2), vacuum=10.0)

slab.calc = calculo
e_slab = slab.get_potential_energy()
f_slab = slab.get_forces()

molecule = Atoms('2N', positions=[(0.0, 0.0, 0.0), (0.0, 0.0, d)])
molecule.calc = calculo
e_N2 = molecule.get_potential_energy()
f_N2 = molecule.get_forces()

add_adsorbate(slab, molecule, h, 'ontop')
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)
dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.0005)

print('Adsorption energy:', e_slab + e_N2 - slab.get_potential_energy())

print(f"Initial slab forces := {f_slab}")
print(f"Initial N2 forces := {f_N2}")

from ase.io import read
from ase.visualize import view

traj = read('N2Cu.traj', ':')   # dos puntos = leer toda la trayectoria
view(traj)                     # ver toda la trayectoria
view(traj[-1])                 # sólo la estructura final

Using Materials Project MACE for MACECalculator with C:\Users\danie\.cache\mace/20231210mace128L0_energy_epoch249model
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using head Default out of ['Default']


c:\Users\danie\AppData\Local\Programs\Python\Python312\Lib\site-packages\mace\calculators\mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 16:04:02     -133.229211       4.3379
BFGSLineSearch:    1[  2] 16:04:03     -133.278030       1.1596
BFGSLineSearch:    2[  3] 16:04:03     -133.297810       0.2362
BFGSLineSearch:    3[  4] 16:04:03     -133.303037       0.1009
BFGSLineSearch:    4[  5] 16:04:03     -133.303148       0.0163
BFGSLineSearch:    5[  6] 16:04:03     -133.303149       0.0010
BFGSLineSearch:    6[  8] 16:04:04     -133.303149       0.0000
Adsorption energy: 0.409857082697215
Initial slab forces := [[-5.77608707e-16 -2.74519990e-16  2.18665912e-01]
 [ 5.81945516e-17  1.24802512e-15  2.18665912e-01]
 [-1.18557508e-16 -1.39428399e-16  2.18665912e-01]
 [-3.89770681e-17 -8.01225405e-17  2.18665912e-01]
 [ 6.14130622e-16  1.74491498e-15  2.18665912e-01]
 [ 1.52059355e-17  9.60928385e-16  2.18665912e-01]
 [ 4.97919848e-16  1.22406425e-15  2.18665912e-01]
 [ 7.59564937e-16  1.49066957e-15  2.18665912e-01]
 [-5.69206141e-17  

<Popen: returncode: None args: ['c:\\Users\\danie\\AppData\\Local\\Programs\...>

### Atoms
The `Atoms` object is a collection of atoms. Here is how to define a N2 molecule by directly specifying the position of two nitrogen atoms:

In [7]:
from ase import Atoms
d = 1.10
molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])

You can also build crystals using, for example, the lattice module which returns Atoms objects corresponding to common crystal structures. Let us make a Cu (111) surface: 

In [8]:
from ase.build import fcc111
slab = fcc111('Cu', size=(4,4,2), vacuum=10.0)

### Adding calculator
In this overview we use the effective medium theory (EMT) calculator, as it is very fast and hence useful for getting started.

We can attach a calculator to the previously created Atoms objects:

In [9]:
from ase.calculators.emt import EMT
slab.calc = EMT()
molecule.calc = EMT()

and use it to calculate the total energies for the systems by using the get_potential_energy() method from the Atoms class:

In [10]:
e_slab = slab.get_potential_energy()
e_N2 = molecule.get_potential_energy()

### Structure relaxation
Let’s use the QuasiNewton minimizer to optimize the structure of the N2 molecule adsorbed on the Cu surface. First add the adsorbate to the Cu slab, for example in the on-top position:

In [11]:
h = 1.85
add_adsorbate(slab, molecule, h, 'ontop')

In order to speed up the relaxation, let us keep the Cu atoms fixed in the slab by using FixAtoms from the constraints module. Only the N2 molecule is then allowed to relax to the equilibrium structure:

In [12]:
from ase.constraints import FixAtoms
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)

Now attach the QuasiNewton minimizer to the system and save the trajectory file. Run the minimizer with the convergence criteria that the force on all atoms should be less than some fmax:

In [13]:
from ase.optimize import QuasiNewton
dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 15:33:57       11.689927       1.0797


BFGSLineSearch:    1[  2] 15:33:57       11.670814       0.4090
BFGSLineSearch:    2[  4] 15:33:57       11.625880       0.0409


True

### Input-output
Writing the atomic positions to a file is done with the write() function:

In [14]:
from ase.io import write
write('slab.xyz', slab)

c:\Users\danie\AppData\Local\Programs\Python\Python312\Lib\site-packages\ase\io\extxyz.py:318: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


In [15]:
from ase.io import read
slab_from_file = read('slab.xyz')

In [15]:
read('slab.xyz')      # last configuration
read('slab.xyz', -1)  # same as above
read('slab.xyz', 0)   # first configuration

Atoms(symbols='Cu32N2', pbc=[True, True, False], cell=[[10.210621920333747, 0.0, 0.0], [5.105310960166873, 8.842657971447272, 0.0], [0.0, 0.0, 22.08423447177455]], tags=..., constraint=FixAtoms(indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]), calculator=SinglePointCalculator(...))

In [18]:
from ase.visualize import view
view(slab)

<Popen: returncode: None args: ['c:\\Users\\danie\\AppData\\Local\\Programs\...>

In [19]:
from ase.md.verlet import VelocityVerlet
from ase import units
dyn = VelocityVerlet(molecule, timestep=1.0 * units.fs)
for i in range(10):
    pot = molecule.get_potential_energy()
    kin = molecule.get_kinetic_energy()
    print('%2d: %.5f eV, %.5f eV, %.5f eV' % (i, pot + kin, pot, kin))
    dyn.run(steps=20)

 0: -16.41484 eV, -16.41484 eV, 0.00000 eV
 1: -16.41489 eV, -16.41498 eV, 0.00008 eV
 2: -16.41487 eV, -16.41526 eV, 0.00039 eV
 3: -16.41493 eV, -16.41568 eV, 0.00075 eV
 4: -16.41494 eV, -16.41645 eV, 0.00151 eV
 5: -16.41499 eV, -16.41701 eV, 0.00202 eV
 6: -16.41505 eV, -16.41829 eV, 0.00324 eV
 7: -16.41508 eV, -16.41886 eV, 0.00378 eV
 8: -16.41518 eV, -16.42057 eV, 0.00538 eV
 9: -16.41519 eV, -16.42106 eV, 0.00587 eV


In [40]:
from ase.io import read
from ase.visualize import view

traj = read('N2Cu.traj', ':')   # dos puntos = leer toda la trayectoria
view(traj)                     # ver toda la trayectoria
view(traj[-1])                 # sólo la estructura final


<Popen: returncode: None args: ['c:\\Users\\danie\\AppData\\Local\\Programs\...>

In [31]:
from ase.build import bulk
cu = bulk('Cu', 'fcc', a=3.6)
cu.calc = EMT(); print("EMT Cu bulk:", cu.get_potential_energy()/len(cu))
cu.calc = calculo; print("MACE Cu bulk:", cu.get_potential_energy()/len(cu))


EMT Cu bulk: -0.006688768685791047
MACE Cu bulk: -4.089824886855758


In [32]:
N2 = Atoms('2N', positions=[(0,0,0),(0,0,1.1)], cell=[10,10,10], pbc=True)
N2.calc = EMT(); print("EMT N2:", N2.get_potential_energy())
N2.calc = calculo; print("MACE N2:", N2.get_potential_energy())


EMT N2: 0.440343573035614
MACE N2: -16.414844604929694


In [ ]:
from ase.io.trajectory import Trajectory
from ase.optimize import QuasiNewton

dyn = QuasiNewton(slab)

traj = Trajectory('N2Cu_MACE.traj', 'w', slab)
dyn.attach(traj.write, interval=1)   # escribe cada paso

dyn.run(fmax=0.05, steps=500)


                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 15:57:15     -133.303148       0.0163


True

Estas primeras simulaciones representaron un primer acercamiento a la libreria ASE, en este caso se simulo una dinamica de adsorción, donde se intentaba observar si una placa de cobre adsorbia una molecula de $N_2$, al emplear EMT se obtuvo "Adsorption energy: 0.32351942231763964" usando mace_mp se obtuvo "Adsorption energy: 0.409857082697215", lo importante aca es notar que la $E_{ads}$ es mayor qué cero, de donde el sistema prefiere alejar la placa de la molecula, es decir una adsorción desfavorable, esto lo evidenciamos al correr las simulaciones con el método view, donde la distancia final de la molecula resulta mayor que la distancia inicial.

### Atoms and calculators

Ahora pasemos de página y veamos un poco de la estructura básica de la libreria.